In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../data/서울시_혼잡도_202212.csv", encoding='cp949')

# 불필요한 컬럼 제거(연변, 역번호)
data = data.loc[:,~data.columns.isin(["연번", "역번호"])]

# 평일과 공휴일로 데이터 구분
data_week = data[data["요일구분"] == "평일"]
data_holiday = data[(data["요일구분"] == "공휴일") | (data["요일구분"] == "토요일")]

In [3]:
data_week.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 552 entries, 0 to 551
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    552 non-null    object 
 1   호선      552 non-null    int64  
 2   출발역     552 non-null    object 
 3   상하구분    552 non-null    object 
 4   5시30분   552 non-null    float64
 5   6시00분   552 non-null    float64
 6   6시30분   552 non-null    float64
 7   7시00분   552 non-null    float64
 8   7시30분   552 non-null    float64
 9   8시00분   552 non-null    float64
 10  8시30분   552 non-null    float64
 11  9시00분   552 non-null    float64
 12  9시30분   552 non-null    float64
 13  10시00분  552 non-null    float64
 14  10시30분  552 non-null    float64
 15  11시00분  552 non-null    float64
 16  11시30분  552 non-null    float64
 17  12시00분  552 non-null    float64
 18  12시30분  552 non-null    float64
 19  13시00분  552 non-null    float64
 20  13시30분  552 non-null    float64
 21  14시00분  552 non-null    float64
 22  14

In [4]:
data_holiday.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1106 entries, 552 to 1657
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    1106 non-null   object 
 1   호선      1106 non-null   int64  
 2   출발역     1106 non-null   object 
 3   상하구분    1106 non-null   object 
 4   5시30분   1070 non-null   float64
 5   6시00분   1096 non-null   float64
 6   6시30분   1096 non-null   float64
 7   7시00분   1096 non-null   float64
 8   7시30분   1096 non-null   float64
 9   8시00분   1096 non-null   float64
 10  8시30분   1096 non-null   float64
 11  9시00분   1096 non-null   float64
 12  9시30분   1096 non-null   float64
 13  10시00분  1096 non-null   float64
 14  10시30분  1096 non-null   float64
 15  11시00분  1096 non-null   float64
 16  11시30분  1096 non-null   float64
 17  12시00분  1096 non-null   float64
 18  12시30분  1096 non-null   float64
 19  13시00분  1096 non-null   float64
 20  13시30분  1096 non-null   float64
 21  14시00분  1096 non-null   float64
 22

In [5]:
# 6시에 없는 역은 모든 시간에 데이터가 없음 -> 삭제
data_holiday = data_holiday.drop(data[data["6시00분"].isna()].index)

# 추가로 23시 30분에도 없는 역 존재 -> 삭제
data_holiday = data_holiday.drop(data_holiday[data_holiday["23시30분"].isna()].index)


# 5시 30분, 00시 00분, 00시 30분은 대다수 없고, 다른 시간으로 분석해도 충분히 값이 나올거라 생각 -> 열 자체를 삭제
data_holiday = data_holiday.drop(["5시30분", "00시00분", "00시30분"], axis = 1)

data_holiday.isna().sum()

요일구분      0
호선        0
출발역       0
상하구분      0
6시00분     0
6시30분     0
7시00분     0
7시30분     0
8시00분     0
8시30분     0
9시00분     0
9시30분     0
10시00분    0
10시30분    0
11시00분    0
11시30분    0
12시00분    0
12시30분    0
13시00분    0
13시30분    0
14시00분    0
14시30분    0
15시00분    0
15시30분    0
16시00분    0
16시30분    0
17시00분    0
17시30분    0
18시00분    0
18시30분    0
19시00분    0
19시30분    0
20시00분    0
20시30분    0
21시00분    0
21시30분    0
22시00분    0
22시30분    0
23시00분    0
23시30분    0
dtype: int64

In [6]:
# 상하선, 내외선 결합(평균)
data_week = data_week.groupby(["요일구분", "호선","출발역"]).agg("mean")
data_week = data_week.reset_index()

data_holiday = data_holiday.groupby(["요일구분", "호선","출발역"]).agg("mean")
data_holiday = data_holiday.reset_index()

In [7]:
# 공휴일의 경우, 토요일과 공휴일 결합
data_holiday = data_holiday.groupby(["호선", "출발역"]).agg("mean")
data_holiday = data_holiday.reset_index()
data_holiday

,호선,출발역,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,9시30분,...,19시00분,19시30분,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분
0,1,동대문,9.475,8.850,9.800,12.450,14.800,17.850,20.250,23.975,...,21.250,20.975,18.150,17.425,18.425,20.175,16.875,14.075,12.350,13.125
1,1,동묘앞,9.350,8.800,8.850,12.625,14.525,17.025,19.525,22.750,...,19.500,19.825,17.225,14.700,17.275,18.350,14.925,12.050,11.150,11.675
2,1,서울역,10.125,9.375,8.900,9.500,16.400,16.575,19.500,24.275,...,28.050,24.250,27.850,21.000,24.925,22.850,19.450,18.050,14.750,8.650
3,1,시청,8.200,9.400,8.825,9.600,13.650,15.325,18.050,23.275,...,26.650,23.750,26.200,20.950,22.150,21.250,18.925,16.250,13.275,11.800
4,1,신설동,11.925,8.725,9.350,11.300,16.725,16.700,19.050,21.100,...,18.350,18.800,20.275,13.950,15.775,16.825,19.225,13.025,9.525,11.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,8,신흥,3.900,3.850,3.700,4.800,6.200,7.400,8.375,8.200,...,6.350,5.225,4.550,4.750,5.675,4.800,4.200,4.225,3.025,3.075
275,8,암사,3.375,2.775,3.800,4.250,6.275,6.450,8.150,7.150,...,2.525,2.350,2.550,2.125,1.850,1.750,1.450,1.075,0.925,0.650
276,8,잠실,15.050,11.925,15.000,14.100,21.025,22.475,27.025,28.750,...,45.400,36.425,37.025,42.975,41.750,41.775,44.050,30.875,25.275,19.150
277,8,장지,13.550,11.750,12.875,16.000,22.750,25.400,30.650,29.875,...,30.200,26.525,25.550,27.100,33.275,27.775,27.975,21.200,17.275,11.975


In [8]:
# 불필요한 컬럼 제거(요일구분)
data_week = data_week.loc[:,~data_week.columns.isin(["요일구분"])]
data_holiday = data_holiday.loc[:,~data_holiday.columns.isin(["요일구분"])]

# 소수점 통일
data_week = round(data_week, 2)
data_holiday = round(data_holiday, 2)

In [9]:
data_week

,호선,출발역,5시30분,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,1,동대문,10.50,13.35,14.15,24.40,44.65,51.35,53.40,39.75,...,23.30,20.10,25.05,21.30,21.05,17.50,14.90,11.85,5.80,2.55
1,1,동묘앞,9.85,13.85,14.30,25.25,43.50,52.65,46.65,33.80,...,21.45,18.50,22.30,19.20,17.50,18.55,14.20,10.60,5.00,0.25
2,1,서울역,9.40,14.55,15.65,26.15,47.30,53.90,53.65,40.75,...,25.45,26.60,26.35,22.15,26.45,21.65,13.75,10.30,8.65,0.60
3,1,시청,7.80,12.25,14.65,22.50,38.90,45.35,41.45,35.15,...,24.80,26.55,26.45,23.00,24.35,20.30,15.15,10.80,6.75,1.80
4,1,신설동,13.65,16.30,17.95,22.55,38.75,48.15,39.80,32.25,...,20.05,16.40,22.05,18.65,20.15,17.60,13.70,10.00,6.50,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,8,신흥,11.15,6.85,8.40,11.40,13.90,19.80,16.65,12.80,...,5.75,6.00,5.85,5.90,5.55,5.20,3.85,2.95,1.45,1.20
272,8,암사,8.15,7.40,11.75,16.90,20.20,24.65,18.50,16.40,...,2.35,2.20,2.35,1.75,2.05,1.20,0.90,0.85,0.35,0.35
273,8,잠실,17.35,28.05,34.30,40.60,54.05,73.55,65.75,53.55,...,48.65,47.10,53.75,51.30,54.00,42.35,33.10,29.00,14.50,7.55
274,8,장지,29.20,25.75,33.55,46.00,62.20,79.10,77.40,63.80,...,32.25,34.60,36.45,33.80,33.80,27.55,20.15,15.90,7.40,4.70


In [10]:
data_holiday

,호선,출발역,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,9시30분,...,19시00분,19시30분,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분
0,1,동대문,9.48,8.85,9.80,12.45,14.80,17.85,20.25,23.98,...,21.25,20.98,18.15,17.42,18.42,20.18,16.88,14.08,12.35,13.12
1,1,동묘앞,9.35,8.80,8.85,12.62,14.52,17.02,19.52,22.75,...,19.50,19.83,17.23,14.70,17.27,18.35,14.92,12.05,11.15,11.68
2,1,서울역,10.12,9.38,8.90,9.50,16.40,16.58,19.50,24.28,...,28.05,24.25,27.85,21.00,24.92,22.85,19.45,18.05,14.75,8.65
3,1,시청,8.20,9.40,8.82,9.60,13.65,15.32,18.05,23.28,...,26.65,23.75,26.20,20.95,22.15,21.25,18.92,16.25,13.27,11.80
4,1,신설동,11.92,8.72,9.35,11.30,16.73,16.70,19.05,21.10,...,18.35,18.80,20.27,13.95,15.78,16.82,19.23,13.02,9.52,11.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,8,신흥,3.90,3.85,3.70,4.80,6.20,7.40,8.38,8.20,...,6.35,5.22,4.55,4.75,5.68,4.80,4.20,4.23,3.02,3.08
275,8,암사,3.38,2.78,3.80,4.25,6.28,6.45,8.15,7.15,...,2.52,2.35,2.55,2.12,1.85,1.75,1.45,1.08,0.92,0.65
276,8,잠실,15.05,11.92,15.00,14.10,21.02,22.48,27.02,28.75,...,45.40,36.42,37.02,42.97,41.75,41.78,44.05,30.88,25.28,19.15
277,8,장지,13.55,11.75,12.88,16.00,22.75,25.40,30.65,29.88,...,30.20,26.52,25.55,27.10,33.28,27.78,27.98,21.20,17.27,11.98


In [11]:
data_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      276 non-null    int64  
 1   출발역     276 non-null    object 
 2   5시30분   276 non-null    float64
 3   6시00분   276 non-null    float64
 4   6시30분   276 non-null    float64
 5   7시00분   276 non-null    float64
 6   7시30분   276 non-null    float64
 7   8시00분   276 non-null    float64
 8   8시30분   276 non-null    float64
 9   9시00분   276 non-null    float64
 10  9시30분   276 non-null    float64
 11  10시00분  276 non-null    float64
 12  10시30분  276 non-null    float64
 13  11시00분  276 non-null    float64
 14  11시30분  276 non-null    float64
 15  12시00분  276 non-null    float64
 16  12시30분  276 non-null    float64
 17  13시00분  276 non-null    float64
 18  13시30분  276 non-null    float64
 19  14시00분  276 non-null    float64
 20  14시30분  276 non-null    float64
 21  15시00분  276 non-null    float64
 22  15

In [12]:
data_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      279 non-null    int64  
 1   출발역     279 non-null    object 
 2   6시00분   279 non-null    float64
 3   6시30분   279 non-null    float64
 4   7시00분   279 non-null    float64
 5   7시30분   279 non-null    float64
 6   8시00분   279 non-null    float64
 7   8시30분   279 non-null    float64
 8   9시00분   279 non-null    float64
 9   9시30분   279 non-null    float64
 10  10시00분  279 non-null    float64
 11  10시30분  279 non-null    float64
 12  11시00분  279 non-null    float64
 13  11시30분  279 non-null    float64
 14  12시00분  279 non-null    float64
 15  12시30분  279 non-null    float64
 16  13시00분  279 non-null    float64
 17  13시30분  279 non-null    float64
 18  14시00분  279 non-null    float64
 19  14시30분  279 non-null    float64
 20  15시00분  279 non-null    float64
 21  15시30분  279 non-null    float64
 22  16

In [13]:
set(data_holiday.출발역) - set(data_week.출발역)

{'별내별가람', '오남', '잠실새내', '진접'}

In [14]:
# 총 지하철역(평일)
len(set(data_week.출발역))

239

In [15]:
# 호선별 지하철역(평일)
l = [len(set(data_week[data_week.호선 == i].출발역)) for i in range(1, 9)]
[print("{}호선 : {}개".format(i, l[i-1])) for i in range(1, 9)]

1호선 : 10개
2호선 : 51개
3호선 : 34개
4호선 : 26개
5호선 : 56개
6호선 : 39개
7호선 : 42개
8호선 : 18개


[None, None, None, None, None, None, None, None]

In [16]:
data_week = data_week.rename(columns = {"출발역" : "역명"})
data_holiday = data_holiday.rename(columns = {"출발역" : "역명"})

In [17]:
import re

def preprocess_station_name(station_name):
    station_name = re.sub(r'\([^)]*\)', '', station_name)
    
    if not station_name.endswith('역'):
        station_name += '역'
    
    return station_name

data_week['역명'] = data_week['역명'].apply(preprocess_station_name)
data_holiday['역명'] = data_holiday['역명'].apply(preprocess_station_name)

In [18]:
data_week.역명 = ["이수역" if i == "총신대입구역" else i for i in data_week.역명 ]

In [19]:
data_holiday.역명 = ["이수역" if i == "총신대입구역" else i for i in data_holiday.역명 ]

In [24]:
data_week = data_week.loc[:,~data_week.columns.isin(["호선"])] 
data_week = data_week.groupby("역명").agg(max).reset_index()

In [27]:
data_holiday = data_holiday.loc[:,~data_holiday.columns.isin(["호선"])] 
data_holiday = data_holiday.groupby("역명").agg(max).reset_index()

In [29]:
data_week.to_csv("../data/Seoul_Congestion_week_2022.csv", encoding='cp949')
data_holiday.to_csv("../data/Seoul_Congestion_holiday_2022.csv", encoding='cp949')

In [25]:
data_week

,역명,5시30분,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,9시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,가락시장역,24.25,29.95,39.65,52.90,77.95,99.20,95.75,77.10,64.45,...,42.80,45.05,48.55,44.30,46.15,38.00,28.45,21.35,10.50,6.75
1,가산디지털단지역,41.25,31.00,37.40,49.75,64.40,66.60,51.55,41.90,34.30,...,38.15,37.75,38.90,36.30,35.15,33.30,31.45,18.75,10.30,10.50
2,강남구청역,19.35,35.50,30.25,42.25,58.45,70.70,54.20,50.40,52.55,...,45.30,42.45,48.75,45.50,49.90,37.75,26.40,23.40,11.90,11.25
3,강남역,39.45,23.55,29.25,38.70,54.95,73.90,79.85,68.10,57.00,...,48.55,48.45,61.25,68.80,65.30,39.75,36.85,20.55,19.45,11.00
4,강동구청역,19.60,28.05,36.55,46.30,61.20,81.20,66.60,62.05,45.50,...,31.95,32.10,32.75,30.90,35.40,25.55,22.05,16.05,8.65,5.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,홍제역,29.55,25.25,30.55,42.20,64.05,84.50,55.95,47.60,41.55,...,29.15,29.35,24.95,29.80,24.40,26.65,21.55,14.60,7.90,6.80
235,화곡역,26.90,21.80,22.05,37.40,52.45,58.90,42.75,28.35,25.15,...,21.35,19.80,19.35,17.95,19.70,15.15,11.40,7.25,6.85,4.00
236,화랑대역,5.90,6.95,8.80,13.15,19.60,22.75,17.75,14.75,10.55,...,5.20,5.05,5.55,5.20,5.45,4.70,3.35,2.10,1.25,0.90
237,회현역,16.15,19.25,17.50,26.55,34.25,45.05,45.30,33.70,39.65,...,25.35,23.05,24.60,24.20,29.95,25.95,19.70,13.25,8.95,6.05


In [28]:
data_holiday

,역명,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,9시30분,10시00분,...,19시00분,19시30분,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분
0,가락시장역,15.55,14.02,16.52,18.60,26.22,29.50,33.85,37.90,38.35,...,39.85,33.52,32.83,35.50,39.35,33.75,34.72,29.42,23.10,16.52
1,가산디지털단지역,17.05,15.88,13.72,16.42,23.10,25.48,29.28,31.42,30.35,...,29.75,27.80,25.40,25.00,25.08,27.42,23.50,22.80,22.40,12.20
2,강남구청역,21.22,16.65,17.77,21.10,24.92,27.92,33.35,33.60,31.12,...,37.60,32.20,31.75,34.75,32.45,30.55,30.72,27.40,24.15,16.55
3,강남역,14.48,13.10,13.38,15.52,20.95,26.65,31.90,31.15,35.95,...,39.60,32.50,35.85,41.15,41.38,41.15,41.80,34.85,31.58,21.18
4,강동구청역,13.55,12.10,14.63,14.63,22.22,23.20,30.90,31.85,28.95,...,28.17,25.08,23.20,24.48,25.67,25.25,26.02,18.30,14.55,11.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,홍제역,13.20,9.82,12.22,14.80,23.48,24.70,28.28,32.88,29.68,...,24.48,23.02,21.42,22.95,24.45,23.52,21.78,18.20,16.10,10.05
238,화곡역,10.48,8.90,9.25,12.48,16.25,18.65,21.67,22.65,21.15,...,17.73,14.90,13.72,13.15,13.25,12.42,14.32,11.40,10.18,7.02
239,화랑대역,2.95,3.75,3.65,5.35,7.22,7.68,8.28,8.68,8.65,...,5.48,4.28,4.30,3.82,4.92,4.30,4.15,3.52,2.90,2.42
240,회현역,12.60,10.15,11.25,13.32,20.98,21.92,29.65,29.40,30.70,...,38.02,34.05,34.62,32.12,32.15,28.80,26.92,24.58,27.45,17.62
